In [4]:
import pandas as pd
from datasets import Dataset
from IPython.display import HTML, display

In [5]:
fist_rnd = pd.read_csv("/home/elicer/dataset/personality_RND.csv")

In [6]:
fist_rnd.head()

,dataSet.info.gender,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.raw.text,dataSet.answer.summary.text,age
0,FEMALE,EXPERIENCED,지원자님이 회사에서 시킨 일이 아닌 스스로 연구하거나 개발해 보고 싶은 분야가 있나...,저는 자동차 배터리 부분을 조금 더 연구해 보고 싶습니다. 사실은 저의 주 업무는 ...,자동차 배터리 부분을 좀 더 연구해 보고 싶습니다. 주 업무는 아니었지만 기업에서 ...,54
1,FEMALE,EXPERIENCED,어려웠던 점을 해결한 경험이 있으신가요 만약에 있다면 간단하게 이야기를 해주시기 바랍니다,어 많은 순간들이 어려웠던 것 같은데 저는 뭐 그 어려웠던 점을 해결하는 방식이 좀...,저는 어려웠던 점을 해결하는 방식이 집요함이라고 생각합니다. 그래서 계속 문제를 뻗...,54
2,FEMALE,EXPERIENCED,자신의 생각이 아닌 지원자님의 다른 주변인들이 생각하는 자신의 단점과 장점을 모두 ...,어 주변 사람들이 굉장히 뭐 저의 다 어 단점이라면 어 싫고 좋음이 좀 너무 분명해...,주변 사람들은 저에 대해 단점을 말하려 했지만 저는 직설적으로 말하려 했습니다. 장...,54
3,FEMALE,EXPERIENCED,지원자님 인생을 살면서 무엇인가 하나에 푹 빠져 몰두해 본 적이 있나요 지원자님이 ...,저는 사실 진짜 그동안 학교만 다니고 취업한 이후에는 정말 일만 하는 사람이었거든요...,"취미활동을 어떻게 만들어야 하는지 잘 모르는 사람이었는데, 어느날 바닷가에서 서핑을...",34
4,FEMALE,EXPERIENCED,지원자님의 장점과 단점을 말씀해 주시고 그것을 특화시키고 극복하기 위한 노력도 함께...,제 장점은 시원시원하고 결단력이나 추진력이 있다는 거구요. 단점은 어 성격이 너무 ...,저는 성격이 너무 급하고 말이 빠르다는 단점을 완벽하게 없애는 노력을 하기보다는 취...,34


In [13]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.3 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [8]:
def create_text_column(df, job_value='RND', interview_type_value='personality'):
    """
    주어진 DataFrame에 새로운 'text' 열을 추가하는 함수입니다.

    Parameters:
    df (pandas.DataFrame): 입력 DataFrame
    job_value (str): 'Job' 열에 추가할 값 (기본값: 'RND')
    interview_type_value (str): 'Interview Type' 열에 추가할 값 (기본값: 'personality')

    Returns:
    pandas.DataFrame: 'text' 열이 추가된 DataFrame
    """
    # 필요한 열 선택
    selected_cols = ['dataSet.info.gender', 'dataSet.info.experience', 'dataSet.question.raw.text', 'dataSet.answer.raw.text', 'dataSet.answer.summary.text', 'age']
    selected_df = df[selected_cols]

    # 열 이름 변경
    selected_df.rename(columns = {"dataSet.info.experience": "Work Experience", "dataSet.question.raw.text":"Question","dataSet.answer.raw.text":"Answer", "dataSet.answer.summary.text": "Answer Summary"}, inplace = True)

    # 'Job' 열 추가
    selected_df['Job'] = job_value

    # 'Interview Type' 열 추가
    selected_df['Interview Type'] = interview_type_value

    # 행 순서 랜덤화 및 인덱스 초기화
    selected_df = selected_df.sample(frac=1).reset_index(drop=True)

    # Work Experience 값에 따른 한국어 번역
    selected_df['Work Experience'] = selected_df['Work Experience'].apply(lambda x: '신입' if x == 'NEW' else '경력직' if x == 'EXPERIENCED' else '알 수 없음')

    # Interview Type 값에 따른 한국어 번역
    selected_df['Interview Type'] = selected_df['Interview Type'].apply(lambda x: '인성면접' if x == 'personality' else '직무면접' if x == 'job' else '알 수 없음')

    # Job 값을 한국어로 번역
    selected_df['Job'] = selected_df['Job'].apply(lambda x: x)

    # text 열 생성
    selected_df['text'] =  selected_df['Job'] + ' 직무의 ' + selected_df['Interview Type'] + '에 참가하게 된 ' + selected_df['Work Experience'] + ' 지원자입니다. ' + selected_df['Answer']
    selected_df['text_summary'] =  selected_df['Job'] + ' 직무의 ' + selected_df['Interview Type'] + '에 참가하게 된 ' + selected_df['Work Experience'] + ' 지원자입니다. ' + selected_df['Answer Summary']

    return selected_df

In [9]:
# 데이터프레임 생성
fist_rnd = pd.read_csv("/home/elicer/dataset/personality_RND.csv")
df = create_text_column(fist_rnd, job_value='RND', interview_type_value='personality')

In [10]:
df = df[:100]

In [9]:
import pandas as pd
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration


# kobart v3
tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

# Function to Generate Summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=1026)
    summary_text_ids = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=1.0,
        max_length=500,
        min_length=100,
        num_beams=6,
        repetition_penalty=1.5,
        no_repeat_ngram_size=15,
    )
    summary_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    return summary_text.strip()  # 마지막 부분의 개행 문자 제거

# Generate Summaries for All Texts
df['kobart-summary'] = df['Answer'].apply(generate_summary)

# 개행 문자 제거
df['kobart-summary'] = df['kobart-summary'].str.replace('\n', ' ')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
df['Kobart-summary'] =  df['Job'] + ' 직무의 ' + df['Interview Type'] + '에 참가하게 된 ' + df['Work Experience'] + ' 지원자입니다. ' + df['kobart-summary']

In [14]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

# T5 모델 로드
model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

# 요약 함수
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary.strip()

# 데이터프레임에 요약 결과 추가
df['t5-summary'] = df['Answer'].apply(generate_summary)

# 개행 문자 제거
df['t5-summary'] = df['t5-summary'].str.replace('\n', ' ')


[nltk_data] Downloading package punkt to /home/elicer/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
df['T5-summary'] =  df['Job'] + ' 직무의 ' + df['Interview Type'] + '에 참가하게 된 ' + df['Work Experience'] + ' 지원자입니다. ' + df['t5-summary']

In [16]:
df['T5-summary']

0     RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업에서 가장 중요하게 생...
1     RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학생 시절 화성에 위치한...
2     RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 이전 회사에서 평사원임에도...
3     RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 하면서 가장 어려웠...
4     RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 피엠피는 프로젝트 매니저를...
                            ...                        
95    RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 아무리 좋은 의견과 창의성...
96    RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어렸을 때부터 계속 연습을...
97    RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학교 재학 시절에 많은 ...
98    RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 초등학교 사 학년 때부터 ...
99    RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 아름다운 공간에 가는 것을...
Name: T5-summary, Length: 100, dtype: object

In [11]:
data = df[['text','text_summary','Kobart-summary']]

In [12]:
data.head()

,text,text_summary,Kobart-summary
0,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 남을 잘 따르는 팔로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 넓은 포용성을 가지고...,"RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 팔로워 형으로, 팀장..."
1,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때는 제일 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 할 때는 각자의 의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때 제일 어...
2,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 시간이 남을 때 주로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격인 저는...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격이라 일...
3,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어 네 저는 어 대학 시절...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절에 동아리에서 아...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절 동아리에서 지역...
4,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 해외에 처음 나갔을 때 영...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


# Set your OpenAI API key

In [14]:
# 새로운 열을 추가하고 초기값을 빈 문자열로 설정합니다.
data['openai_chain_question_text'] = ""
data['openai_chain_question_text_summary'] = ""
data['openai_chain_question_kobart_summary'] = ""
data['openai_chain_question_T5_summary'] = ""

for i in range(len(df)):
    # ChatOpenAI 인스턴스 생성
    chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

    # 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional interviewer. Examine the provided context and based on the keyword in the following sentence, create a question that requires an answer not found within the sentence itself. Please format your question with the main points in bullet points and ensure the question ends with a '?'. Additionally, translate your question to Korean."
            ),
            (
                "user",
                "Please make question according to the following request."
                "\nREQUEST:\n"
                "1. make question the main points in bullet points in Korean."
                "2. The last part of the context is '?' It must be this."  
                "3. Look at the following sentence and ask a question based on the keyword in the next sentence but question requires an answer not found within the sentence itself"
                "4. Look at the following sentence and ask a question based on the keyword in the next sentence which  answer cannot be found within the sentence"
                "6. Do NOT translate korean to english"
                "5. if question is not korean, translate to korean"
                "6. print 3 question and end."
                "\n\nCONTEXT: {context}\n\nSUMMARY:",
            ),
        ]
    )

    # 각 열에 대한 텍스트 가져오기
    text = data.loc[i, 'text']
    text_summary = data.loc[i, 'text_summary']
    kobart_summary = data.loc[i, 'Kobart-summary']
    T5_summary = data.loc[i, 'T5-summary']

    # LLM을 사용하여 체인 생성
    chain_text = LLMChain(llm=chat, prompt=prompt)
    chain_text_summary = LLMChain(llm=chat, prompt=prompt)
    chain_kobart_summary = LLMChain(llm=chat, prompt=prompt)
    chain_T5_summary = LLMChain(llm=chat, prompt=prompt)

    # 각 열에 대한 질문 생성
    result_text = chain_text.run(context=text)
    result_text_summary = chain_text_summary.run(context=text_summary)
    result_kobart_summary = chain_kobart_summary.run(context=kobart_summary)
    result_T5_summary = chain_T5_summary.run(context=T5_summary)

    # 생성된 결과를 적절한 열에 저장합니다.
    data.loc[i, 'openai_chain_question_text'] = result_text
    data.loc[i, 'openai_chain_question_text_summary'] = result_text_summary
    data.loc[i, 'openai_chain_question_kobart_summary'] = result_kobart_summary
    data.loc[i, 'openai_chain_question_T5_summary'] = result_T5_summary

/tmp/ipykernel_44431/3021012019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['openai_chain_kobart_question_text'] = ""
/tmp/ipykernel_44431/3021012019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['openai_chain_kobart_question_text_summary'] = ""
/tmp/ipykernel_44431/3021012019.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [15]:
question_data = data[['openai_chain_question_text','openai_chain_question_text_summary','openai_chain_question_kobart_summary','openai_chain_question_T5_summary']]

In [16]:
question_data.head()

,openai_chain_question_text,openai_chain_question_text_summary,openai_chain_question_kobart_summary
0,1. 팔로워 형태의 리더십에 대해 어떻게 생각하시나요?\n2. 어떤 상황에서 리더 ...,1. RND 직무의 인성면접에 참가한 신입 지원자입니다. 한 명 한 명의 개개인을 ...,1. RND 직무의 인성면접에 참가한 경험이 있는가요?\n2. 지원자로서 팀장을 서...
1,1. 협업 시 기준점 설정의 중요성에 대해 어떻게 생각하시나요?\n2. 협업에서의 ...,1. 협업 시 원칙을 세우는 것이 중요한가요?\n2. 협업 시 양쪽 다 이익을 낼 ...,1. 협업 시 어려운 점은 무엇입니까?\n2. 협업을 위해 필요한 기준점은 무엇이라...
2,1. 저의 시간 관리 방법은 무엇입니까?\n2. 일정을 정리하면 어떤 이점을 느끼십...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 설명했...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 말했습...
3,1. 어떤 프로젝트를 통해 아이들에게 교육적인 지식을 전달했나요?\n2. 어떤 효과...,1. 대학 시절에 어떤 종류의 요리 프로그램을 아동들을 대상으로 진행했나요?\n2....,1. 대학 시절 동아리에서 지역아동센터 아동들을 대상으로 요리 프로그램을 진행한 경...
4,1. 해외에서의 어려움을 극복하기 위해 어떤 노력을 했나요?\n2. 어떤 결과를 얻...,1. 해외 국제학교에서 영어를 배우며 노력한 경험에 대해 어떤 동기가 있었나요?\n...,1. 해외에서의 영어 학습 경험에 대해 어떤 도전이 있었나요?\n2. 어떤 마음가짐...


### GPTScore를 통한 성능 평가

In [13]:
import pandas as pd
data = pd.read_csv('/home/elicer/model/inno/summary/new_df.csv')
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [14]:
data.head()

,text,text_summary,Kobart-summary,T5-summary,openai_chain_question_text,openai_chain_question_text_summary,openai_chain_question_kobart_summary,openai_chain_question_T5_summary
0,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 남을 잘 따르는 팔로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 넓은 포용성을 가지고...,"RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 팔로워 형으로, 팀장...",RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업에서 가장 중요하게 생...,1. 팔로워 형태의 리더십에 대해 어떻게 생각하시나요?\n2. 어떤 상황에서 리더 ...,1. RND 직무의 인성면접에 참가한 신입 지원자입니다. 한 명 한 명의 개개인을 ...,1. RND 직무의 인성면접에 참가한 경험이 있는가요?\n2. 지원자로서 팀장을 서...,1. 협업에서 가장 중요하게 생각하는 것은 무엇입니까?\n2. 인성면접에서 어떤 키...
1,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때는 제일 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 할 때는 각자의 의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때 제일 어...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학생 시절 화성에 위치한...,1. 협업 시 기준점 설정의 중요성에 대해 어떻게 생각하시나요?\n2. 협업에서의 ...,1. 협업 시 원칙을 세우는 것이 중요한가요?\n2. 협업 시 양쪽 다 이익을 낼 ...,1. 협업 시 어려운 점은 무엇입니까?\n2. 협업을 위해 필요한 기준점은 무엇이라...,1. 화성 공장 견학 경험에 대해 어떤 부분이 가장 기억에 남았나요?\n2. 생산과...
2,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 시간이 남을 때 주로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격인 저는...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격이라 일...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 이전 회사에서 평사원임에도...,1. 저의 시간 관리 방법은 무엇입니까?\n2. 일정을 정리하면 어떤 이점을 느끼십...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 설명했...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 말했습...,1. 이전 회사에서 어떤 업무를 맡았나요?\n2. 어떤 경험을 통해 목표를 달성했나...
3,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어 네 저는 어 대학 시절...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절에 동아리에서 아...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절 동아리에서 지역...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 하면서 가장 어려웠...,1. 어떤 프로젝트를 통해 아이들에게 교육적인 지식을 전달했나요?\n2. 어떤 효과...,1. 대학 시절에 어떤 종류의 요리 프로그램을 아동들을 대상으로 진행했나요?\n2....,1. 대학 시절 동아리에서 지역아동센터 아동들을 대상으로 요리 프로그램을 진행한 경...,1. 동료와의 의견 조율이 어려웠던 경험을 어떻게 극복했나요?\n2. 상대방의 의견...
4,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 해외에 처음 나갔을 때 영...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 피엠피는 프로젝트 매니저를...,1. 해외에서의 어려움을 극복하기 위해 어떤 노력을 했나요?\n2. 어떤 결과를 얻...,1. 해외 국제학교에서 영어를 배우며 노력한 경험에 대해 어떤 동기가 있었나요?\n...,1. 해외에서의 영어 학습 경험에 대해 어떤 도전이 있었나요?\n2. 어떤 마음가짐...,1. RND 직무의 인성면접에 참가한 신입 지원자가 피엠피에서 어떤 성과를 얻었는지...


In [8]:
import time
import sys
from transformers import GPT2Tokenizer
import openai

class GPT3Model(object):

    def __init__(self, model_name, api_key, logger=None):
        self.model_name = model_name
        try:
            openai.api_key = api_key
        except Exception:
            pass
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
        self.logger=logger

    def do_inference(self, input, output, max_length=2048):
        losses = []
        data = input + output

        response = self.gpt3(data)
        out = response["choices"][0]

        assert input + output == out["text"]
        i = 0 
        for idx, offset in enumerate(out['logprobs']['text_offset']):
            if offset >= len(input)-1:
                i = idx
                break

        if i == 0:
            i += 1

        print('eval text', out['logprobs']['tokens'][i: -1])
        loss = -sum(out['logprobs']["token_logprobs"][i:-1]) # ignore the last '.'
        avg_loss = loss / (len(out['logprobs']['text_offset']) - i-1) # 1 is the last '.'
        print('avg_loss: ', avg_loss)
        losses.append(avg_loss)

        return avg_loss


    def gpt3(self, prompt, max_len=0, temp=0, num_log_probs=0, echo=True, n=None):
        response = None
        received = False
        while not received:
            try:
                response = openai.Completion.create(engine=self.model_name,
                                                    prompt=prompt,
                                                    max_tokens=max_len,
                                                    temperature=temp,
                                                    logprobs=num_log_probs,
                                                    echo=echo,
                                                    stop='\n',
                                                    n=n)
                print('prompt: ',prompt)
                received = True
            except:
                error = sys.exc_info()[0]
                if error == openai.error.InvalidRequestError:
                    # something is wrong: e.g. prompt too long
                    print(f"InvalidRequestError\nPrompt passed in:\n\n{prompt}\n\n")
                    assert False
                print("API error:", error)
                time.sleep(1)
        return response


def gpt3score(input, output, gpt3model=None,api_key=None):
    metaicl_model = GPT3Model(gpt3model, api_key)
    avg_loss = metaicl_model.do_inference(input, output)
    score = -avg_loss
    return score


def calculate_gpt3_scores(data, api_key, gpt3_model="davinci-002"):
    
    openai.api_key = api_key
    
    data['text_gpt3_score'] = 0 
    data['text_summary_gpt3_score'] = 0
    data['kobart_summary_gpt3_score'] = 0 
    data['T5_summary_gpt3_score'] = 0

    for idx in data.index:
        text = data.loc[idx, 'text']  
        question_text = data.loc[idx, 'openai_chain_question_text']
        
        text_summary = data.loc[idx, 'text_summary']
        question_text_summary = data.loc[idx, 'openai_chain_question_text_summary']  
        
        kobart_summary = data.loc[idx, 'Kobart-summary']
        question_kobart_summary = data.loc[idx, 'openai_chain_question_kobart_summary']
        
        T5_summary = data.loc[idx, 'T5-summary']
        question_T5_summary = data.loc[idx, 'openai_chain_question_T5_summary']
            
        data.at[idx, 'text_gpt3_score'] = gpt3score(text, question_text, gpt3_model, api_key)  
        data.at[idx, 'text_summary_gpt3_score'] = gpt3score(text_summary, question_text_summary, gpt3_model, api_key)
        data.at[idx, 'kobart_summary_gpt3_score'] = gpt3score(kobart_summary, question_kobart_summary, gpt3_model, api_key) 
        data.at[idx, 'T5_summary_gpt3_score'] = gpt3score(T5_summary, question_T5_summary, gpt3_model, api_key)
        
    return data


In [15]:
calculate_gpt3_scores(data, api_key=openai_api_key, gpt3_model="davinci-002")

prompt:  RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 남을 잘 따르는 팔로워 형입니다. 물론 상황에 따라 리더의 역할을 수행해야 한다면 기꺼이 리더가 되는 멀티 플레이어가 되고자 하지만 기본적으로는 팀장을 서포트 해주는 역할을 주로 맡고 있습니다. 일단 저는 뒤에서 보조해 주는 것을 잘합니다. 리더나 팀원들이 필요로 하는 자료나 레퍼런스를 가져오거나 멀리서 상황을 객관적으로 지켜보고 정리하는 것을 좋아합니다. 리더는 많은 책임을 갖고 모두를 이끌어야 합니다. 저는 그런 넓은 포용성을 가지고 있기보다는 한 명 한 명의 개개인을 세심하게 관찰하고 커뮤니케이션을 하는 것을 더 잘하는 편입니다. 따라서 저는 모든 팀원들과 화합하는 팔로워 형이라고 소개하겠습니다.1. 팔로워 형태의 리더십에 대해 어떻게 생각하시나요?
2. 어떤 상황에서 리더 역할을 맡아본 경험이 있으신가요?
3. 팀원들과의 화합을 위해 어떤 노력을 기울이고 계신가요? 

한국어로 번역:
1. 팔로워 형태의 리더십에 대해 어떻게 생각하시나요?
2. 어떤 상황에서 리더 역할을 맡아본 경험이 있으신가요?
3. 팀원들과의 화합을 위해 어떤 노력을 기울이고 계신가요?
eval text ['.', '1', '.', 'bytes: \\xed\\x8c', 'bytes:\\x94', '로', 'bytes:\\xec\\x9b', 'bytes:\\x8c', 'bytes: \\xed\\x98', 'bytes:\\x95', '태', '의', ' 리', 'bytes:\\xeb\\x8d', 'bytes:\\x94', 'bytes:\\xec\\x8b', 'bytes:\\xad', '에', ' 대', '해', 'bytes: \\xec\\x96', 'bytes:\\xb4', 'bytes:\\xeb', 'bytes:\\x96', 'bytes:\\xbb', '게', ' 생', 'bytes:\\xea\\xb0', 'bytes:\\x81', '하', '시', '나', '요', '?\n', '2', '.', 'byte

/tmp/ipykernel_1343/1880145789.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.6848009238957418' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[idx, 'text_gpt3_score'] = gpt3score(text, question_text, gpt3_model, api_key)


prompt:  RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 넓은 포용성을 가지고 있기보다는 한 명 한 명의 개개인을 세심하게 관찰하고 커뮤니케이션을 하는 것을 더 잘하는 편입니다. 따라서 저는 모든 팀원들과 화합하는 팔로워 형이라고 소개하겠습니다.1. RND 직무의 인성면접에 참가한 신입 지원자입니다. 한 명 한 명의 개개인을 세심하게 관찰하고 커뮤니케이션을 잘하는 편이라고 소개했습니다. 어떤 상황에서 세심한 관찰이 도움이 되었나요?
2. 지원자는 모든 팀원들과 화합하는 팔로워 형이라고 소개했습니다. 어떻게하여 팀원들과의 화합을 이루는데 기여할 수 있을까요?
3. 지원자는 넓은 포용성보다는 한 명 한 명의 개개인을 세심하게 관찰하고 커뮤니케이션을 잘하는 편이라고 했습니다. 이러한 세심함이 어떻게 업무 수행에 도움이 될 수 있을까요?
eval text ['.', '1', '.', ' R', 'ND', 'bytes: \\xec\\xa7', 'bytes:\\x81', 'bytes:\\xeb', 'bytes:\\xac\\xb4', '의', ' 인', '성', '면', 'bytes:\\xec\\xa0', 'bytes:\\x91', '에', 'bytes: \\xec\\xb0', 'bytes:\\xb8', '가', '한', 'bytes: \\xec\\x8b', 'bytes:\\xa0', '입', ' 지', '원', '자', '입니다', '.', ' 한', 'bytes: \\xeb\\xaa', 'bytes:\\x85', ' 한', 'bytes: \\xeb\\xaa', 'bytes:\\x85', '의', ' 개', '개', '인', '을', 'bytes: \\xec\\x84', 'bytes:\\xb8', 'bytes:\\xec\\x8b', 'bytes:\\xac', '하', '게', ' 관', 'bytes:\\xec\\xb0', 'bytes:\\xb0', '하', '고', 'bytes: \\xec\\xbb', 'bytes:\\xa4', 'bytes:\\xe

/tmp/ipykernel_1343/1880145789.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.769346331950938' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[idx, 'text_summary_gpt3_score'] = gpt3score(text_summary, question_text_summary, gpt3_model, api_key)


prompt:  RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 팔로워 형으로, 팀장을 서포트하는 역할을 주로 맡고 있습니다. 리더는 많은 책임을 갖고 모두를 이끌어야 하므로, 저는 모든 팀원들과 화합하는 팔로워 형이라고 소개하겠습니다.                               .             .1. RND 직무의 인성면접에 참가한 경험이 있는가요?
2. 지원자로서 팀장을 서포트하는 역할에서 어떤 어려움을 겪었나요?
3. 팔로워 형으로서 팀원들과의 화합을 이루기 위해 어떤 노력을 기울였나요?
eval text ['1', '.', ' R', 'ND', 'bytes: \\xec\\xa7', 'bytes:\\x81', 'bytes:\\xeb', 'bytes:\\xac\\xb4', '의', ' 인', '성', '면', 'bytes:\\xec\\xa0', 'bytes:\\x91', '에', 'bytes: \\xec\\xb0', 'bytes:\\xb8', '가', '한', ' 경', 'bytes:\\xed', 'bytes:\\x97', 'bytes:\\x98\\xec\\x9d\\xb4', ' 있는', '가', '요', '?\n', '2', '.', ' 지', '원', '자', '로', '서', 'bytes: \\xed', 'bytes:\\x8c\\x80', '장', '을', ' 서', 'bytes:\\xed', 'bytes:\\x8f', 'bytes:\\xac', '트', '하는', 'bytes: \\xec\\x97', 'bytes:\\xad', '할', '에서', 'bytes: \\xec\\x96', 'bytes:\\xb4', 'bytes:\\xeb', 'bytes:\\x96', 'bytes:\\xa4', 'bytes: \\xec\\x96', 'bytes:\\xb4', 'bytes:\\xeb\\xa0', 'bytes:\\xa4', 'bytes:\\xec\\x9b', 'bytes:\\x80', '을', 'bytes: \\xea\\xb2

/tmp/ipykernel_1343/1880145789.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.9496975259708834' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[idx, 'kobart_summary_gpt3_score'] = gpt3score(kobart_summary, question_kobart_summary, gpt3_model, api_key)


prompt:  RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업에서 가장 중요하게 생각하는 것은 배려입니다. 협업에서 소통의 중요성은 두 말하면 잔소리일 정도로 이 협업의 결과에 가장 큰 영향을 미치는 것이 이 소통이라고 할 수 있는데 이 소통에서 가장 중요한 키워드가 배려라고 생각합니다. 협업의 결과에 가장 큰 영향을 미치는 것이 이 소통이라고 할 수 있는데 이 소통에서 가장 중요한1. 협업에서 가장 중요하게 생각하는 것은 무엇입니까?
2. 인성면접에서 어떤 키워드가 가장 중요하다고 생각합니까?
3. 협업에서 소통의 중요성 외에 다른 요소가 있을까요?
eval text ['한', '1', '.', 'bytes: \\xed\\x98', 'bytes:\\x91', 'bytes:\\xec\\x97', 'bytes:\\x85', '에서', ' 가', '장', ' 중', '요', '하', '게', ' 생', 'bytes:\\xea\\xb0', 'bytes:\\x81', '하는', ' 것', '은', 'bytes: \\xeb', 'bytes:\\xac\\xb4', 'bytes:\\xec\\x97', 'bytes:\\x87', '입', '니', 'bytes:\\xea\\xb9', 'bytes:\\x8c', '?\n', '2', '.', ' 인', '성', '면', 'bytes:\\xec\\xa0', 'bytes:\\x91', '에서', 'bytes: \\xec\\x96', 'bytes:\\xb4', 'bytes:\\xeb', 'bytes:\\x96', 'bytes:\\xa4', 'bytes: \\xed', 'bytes:\\x82\\xa4', 'bytes:\\xec\\x9b', 'bytes:\\x8c', '드', '가', ' 가', '장', ' 중', '요', '하', '다', '고', ' 생', 'bytes:\\xea\\xb0', 'bytes:\\x81', 'bytes:\\xed\\x95', 'bytes:\\xa9', '니', 'bytes:\\xea\\xb

/tmp/ipykernel_1343/1880145789.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.9407908728579891' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[idx, 'T5_summary_gpt3_score'] = gpt3score(T5_summary, question_T5_summary, gpt3_model, api_key)


prompt:  RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때는 제일 어려운 점은 서로 갈등이 많이 야기된다는 것입니다. 각자의 의견을 다 내세우다 보면 그 의견 조율이 쉽지 않습니다. 그래서 저는 어떤 협업을 할 때는 그 기준점을 둬야 된다고 생각합니다. 아 양쪽 다 어 합의할 수 있는 기준점 자기들이 에 생각하는 것보다 기대에 못 미치다 하더라도 그게 얼마든지 좀 순응할 수 있는 그런 어떤 기준점을 세워 놓고 그 기준점에 부합하다면은 그렇게 해서 협업을 하는 것이 가장 이상적인 생각이라고 알고 있습니다. 협업은 말 그대로 양쪽 다의 좋은 이익을 날 수 있는 그런 시스템으로 가야지만 잘 이루어진다고 봅니다. 일방적으로 어느 한쪽이 불리하게 에 작용되는 협업은 절대 어 이루어질 수가 없다고 봅니다. 물론 조금의 차이는 있을 수 있습니다. 예를 들어 자본금이 많은 사람과 자본금이 없는 사람이 협업할 때는 자본금이 많은 사람에게 훨씬 더 유리한 쪽으로 흘러갈 수 밖에 없는 구조가 될 수도 있죠. 하지만 그 모두가 어 원칙을 정해 놓고 그 원칙에 따라 어 협업을 해야지만 그 협업은 잘 이루어질 수 있다고 봅니다. 어떤 룰이 정해졌는데 그 룰을 어 무리식 무시하고 진행한다면 절대 그 협업은 음 이루어질 수 없다고 봅니다. 갈등만 유발하고 또는 어떤 안좋은 결과만이 이 생산될 수 있을 수밖에 없습니다. 그래서 협업은 참으로 어 어떤 원칙을 잘 세워서 할 때만이 성공할 수 있다고 생각합니다.1. 협업 시 기준점 설정의 중요성에 대해 어떻게 생각하시나요?
2. 협업에서의 원칙 무시가 협업에 어떤 영향을 미칠 수 있다고 생각하시나요?
3. 협업의 성공을 위해 필요한 원칙은 무엇이라고 생각하시나요? ?
eval text ['.', '1', '.', 'bytes: \\xed\\x98', 'bytes:\\x91', 'bytes:\\xec\\x97', 'bytes:\\x85', ' 시', ' 기', 'bytes:\\xec\\xa4', 'bytes:\\x

,text,text_summary,Kobart-summary,T5-summary,openai_chain_question_text,openai_chain_question_text_summary,openai_chain_question_kobart_summary,openai_chain_question_T5_summary,text_gpt3_score,text_summary_gpt3_score,kobart_summary_gpt3_score,T5_summary_gpt3_score
0,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 남을 잘 따르는 팔로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 넓은 포용성을 가지고...,"RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 팔로워 형으로, 팀장...",RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업에서 가장 중요하게 생...,1. 팔로워 형태의 리더십에 대해 어떻게 생각하시나요?\n2. 어떤 상황에서 리더 ...,1. RND 직무의 인성면접에 참가한 신입 지원자입니다. 한 명 한 명의 개개인을 ...,1. RND 직무의 인성면접에 참가한 경험이 있는가요?\n2. 지원자로서 팀장을 서...,1. 협업에서 가장 중요하게 생각하는 것은 무엇입니까?\n2. 인성면접에서 어떤 키...,-0.684801,-0.769346,-0.949698,-0.940791
1,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때는 제일 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 할 때는 각자의 의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 네 협업을 할 때 제일 어...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학생 시절 화성에 위치한...,1. 협업 시 기준점 설정의 중요성에 대해 어떻게 생각하시나요?\n2. 협업에서의 ...,1. 협업 시 원칙을 세우는 것이 중요한가요?\n2. 협업 시 양쪽 다 이익을 낼 ...,1. 협업 시 어려운 점은 무엇입니까?\n2. 협업을 위해 필요한 기준점은 무엇이라...,1. 화성 공장 견학 경험에 대해 어떤 부분이 가장 기억에 남았나요?\n2. 생산과...,-1.032681,-1.089120,-0.857545,-1.025065
2,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 시간이 남을 때 주로...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격인 저는...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 꼼꼼하지 못한 성격이라 일...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 이전 회사에서 평사원임에도...,1. 저의 시간 관리 방법은 무엇입니까?\n2. 일정을 정리하면 어떤 이점을 느끼십...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 설명했...,1. RND 직무의 인성면접에 참가한 신입 지원자가 꼼꼼하지 못한 성격이라고 말했습...,1. 이전 회사에서 어떤 업무를 맡았나요?\n2. 어떤 경험을 통해 목표를 달성했나...,-0.725217,-0.781066,-0.965945,-0.835392
3,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어 네 저는 어 대학 시절...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절에 동아리에서 아...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학 시절 동아리에서 지역...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 협업을 하면서 가장 어려웠...,1. 어떤 프로젝트를 통해 아이들에게 교육적인 지식을 전달했나요?\n2. 어떤 효과...,1. 대학 시절에 어떤 종류의 요리 프로그램을 아동들을 대상으로 진행했나요?\n2....,1. 대학 시절 동아리에서 지역아동센터 아동들을 대상으로 요리 프로그램을 진행한 경...,1. 동료와의 의견 조율이 어려웠던 경험을 어떻게 극복했나요?\n2. 상대방의 의견...,-0.798230,-0.951017,-0.814279,-0.637892
4,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 해외에 처음 나갔을 때 영...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 제가 인생을 살면서 무언가...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 피엠피는 프로젝트 매니저를...,1. 해외에서의 어려움을 극복하기 위해 어떤 노력을 했나요?\n2. 어떤 결과를 얻...,1. 해외 국제학교에서 영어를 배우며 노력한 경험에 대해 어떤 동기가 있었나요?\n...,1. 해외에서의 영어 학습 경험에 대해 어떤 도전이 있었나요?\n2. 어떤 마음가짐...,1. RND 직무의 인성면접에 참가한 신입 지원자가 피엠피에서 어떤 성과를 얻었는지...,-0.616219,-0.679552,-0.874492,-0.469612
...,...,...,...,...,...,...,...,...,...,...,...,...
95,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어 사실 저는 대학교 때 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 대학교 때 차가워서 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 대학교 때 차가워서 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 아무리 좋은 의견과 창의성...,1. 대학교 때 들었던 차갑다는 이야기에 대해 어떻게 대처했나요?\n2. 지금까지 ...,1. 대학교 때 차가워서 다가가기 힘든 사람이라는 이야기를 많이 들었습니다. 그래서...,1. 대학교 때 차가웠던 신입 지원자가 RND 직무의 인성면접에 참가했습니다. 변화...,1. 팀워크가 왜 RND 직무에서 중요한가요?\n2. 어떻게 팀워크가 창의성을 발휘...,-0.753039,-0.510976,-1.062702,-1.149996
96,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 잘 맞지 않더라도 최선을 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 신입이고 그 일에 적응하는...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 입사 후 근무를 하면서 제...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 어렸을 때부터 계속 연습을...,1. RND 직무의 인성면접에 참가한 신입 지원자입니다. 최선을 다하겠다고 말했습니...,1. RND 직무의 인성면접에 참가한 이유는 무엇인가요?\n2. 신입 지원자로서 어...,1. RND 직무의 인성면접에 참가한 신입 지원자가 자신의 업무에 대해 어떤 태도를...,1. 어떤 직무의 인성면접에 참가하게 된 신입 지원자입니까? \n2. 어릴 때부터 ...,-1.118700,-1.109096,-0.642661,-1.138489
97,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 업무를 하다 보면 상사와의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 업무를 하다 보면 상사와의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 업무를 하다 보면 상사와의...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 대학교 재학 시절에 많은 ...,1. RND 직무의 인성면접에 참가한 신입 지원자가 상사와의 트러블에 대해 생각을 ...,1. RND 직무의 인성면접에 참가한 신입 지원자가 상사와의 트러블에 대해 언급\n...,1. RND 직무의 인성면접에 참가한 신입 지원자가 상사와의 트러블에 대해 생각하고...,1. 대학교 시절 팀 프로젝트에서 리더로 활동한 경험이 있었나요?\n2. 리더로서의...,-0.938307,-0.680122,-0.520471,-0.673825
98,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 결정은 저를 발전시킬 수 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 회사 인턴 생활을 하면서 ...,RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 회사의 인턴 

In [17]:
score = data[['text_gpt3_score','text_summary_gpt3_score','kobart_summary_gpt3_score','T5_summary_gpt3_score']]

In [18]:
score.describe()

,text_gpt3_score,text_summary_gpt3_score,kobart_summary_gpt3_score,T5_summary_gpt3_score
count,100.000000,100.000000,100.000000,100.000000
mean,-0.810736,-0.869800,-0.835815,-0.876327
std,0.208878,0.176016,0.175987,0.178402
min,-1.324720,-1.224736,-1.192063,-1.246366
25%,-0.953480,-0.987180,-0.954070,-1.006376
50%,-0.807057,-0.877099,-0.847857,-0.887659
75%,-0.636163,-0.757260,-0.720820,-0.748216
max,-0.227947,-0.481453,-0.356765,-0.469612


### Prompt를 통한 질문 성능 평가

In [12]:
# ChatOpenAI 인스턴스 생성
chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

# 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant that evaluates the quality of interview questions. Examine the provided context and the generated question, and rate the quality of the question based on the following criteria:"
        ),
        (
            "user",
            "Please evaluates the quality of question according to the following request."
            "\nREQUEST:\n"
            "Clarity: Is the question clearly formulated and easy to understand?"
            "Relevance: Is the question relevant to the context provided?"
            "Openness: Does the question encourage open-ended responses?"
            "Depth: Does the question delve into the topic deeply and thoughtfully?"
            "Originality: Is the question unique and thought-provoking?"
            "Please rate the quality of the following interview question based on the criteria above, using a scale of 1 to 10, with 10 being the highest quality." 
            "Question Quality Score should be placed at the end of the sentence."
            "\n\nContext: {context}\n\nQuestion Quality Score: "
        )
    ]
)

# 각 열에 대한 텍스트 가져오기
for i in range(len(data)):
    text_question = data.loc[i, 'openai_chain_question_text']
    text_summary_question = data.loc[i, 'openai_chain_question_text_summary']
    kobart_summary_question = data.loc[i, 'openai_chain_question_kobart_summary']
    T5_summary_question = data.loc[i, 'openai_chain_question_T5_summary']

    # LLM을 사용하여 체인 생성
    chain_text_question = LLMChain(llm=chat, prompt=prompt)
    chain_text_summary_question = LLMChain(llm=chat, prompt=prompt)
    chain_kobart_summary_question = LLMChain(llm=chat, prompt=prompt)
    chain_T5_summary_question = LLMChain(llm=chat, prompt=prompt)

    # 각 열에 대한 질문 품질 평가
    result_text_quality = chain_text_question.run(context=text_question)
    result_text_summary_quality = chain_text_summary_question.run(context=text_summary_question)
    result_kobart_summary_quality = chain_kobart_summary_question.run(context=kobart_summary_question)
    result_T5_summary_quality = chain_kobart_summary_question.run(context=T5_summary_question)

    # 생성된 결과를 적절한 열에 저장합니다.
    data.loc[i, 'openai_chain_question_text_quality'] = result_text_quality
    data.loc[i, 'openai_chain_question_text_summary_quality'] = result_text_summary_quality
    data.loc[i, 'openai_chain_question_kobart_summary_quality'] = result_kobart_summary_quality
    data.loc[i, 'openai_chain_question_T5_summary_quality'] = result_T5_summary_quality


/home/elicer/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
question_data = data.copy()

In [23]:
question_data.to_csv('/home/elicer/model/inno/summary/question_data.csv')

In [19]:
import re

def extract_quality_score(quality_str):
    """
    주어진 quality 문자열에서 Quality Score를 추출하여 반환합니다.
    소수점 1자리까지 반환합니다.
    
    Args:
        quality_str (str): Quality 평가 문자열
    
    Returns:
        float: Quality Score (소수점 1자리까지)
    """
    if isinstance(quality_str, str):
        # 정규 표현식을 사용하여 'scores:', 'score:', 'Scores:', 'Score:' 등의 패턴을 찾아 숫자 부분을 추출
        match = re.search(r'[Qq]uality [Ss]cores?:?\s*(\d+(\.\d)?)', quality_str)
        if match:
            quality_score = float(match.group(1))
            return round(quality_score, 1)
        else:
            return None
    else:
        return None


question_data['openai_chain_question_text_quality_score'] = question_data['openai_chain_question_text_quality'].apply(extract_quality_score)
question_data['openai_chain_text_summary_quality_score'] = question_data['openai_chain_question_text_summary_quality'].apply(extract_quality_score)
question_data['openai_chain_text_kobart_summary_quality_score'] = question_data['openai_chain_question_kobart_summary_quality'].apply(extract_quality_score)
question_data['openai_chain_text_T5_summary_quality_score'] = question_data['openai_chain_question_T5_summary_quality'].apply(extract_quality_score)



In [20]:
question_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   text                                            100 non-null    object 
 1   text_summary                                    100 non-null    object 
 2   Kobart-summary                                  100 non-null    object 
 3   T5-summary                                      100 non-null    object 
 4   openai_chain_question_text                      100 non-null    object 
 5   openai_chain_question_text_summary              100 non-null    object 
 6   openai_chain_question_kobart_summary            100 non-null    object 
 7   openai_chain_question_T5_summary                100 non-null    object 
 8   openai_chain_question_text_quality              100 non-null    object 
 9   openai_chain_question_text_summary_quality  

In [21]:
question_data[['openai_chain_question_kobart_summary_quality','openai_chain_text_kobart_summary_quality_score','openai_chain_text_T5_summary_quality_score']]

,openai_chain_question_kobart_summary_quality,openai_chain_text_kobart_summary_quality_score,openai_chain_text_T5_summary_quality_score
0,"""Can you provide an example of a challenging s...",8.0,8.0
1,"""What strategies do you believe are essential ...",8.0,8.0
2,"""How do you think the candidate is trying to o...",8.4,8.0
3,Question Quality Score: 8,8.0,8.0
4,"""Can you share a specific challenge you faced ...",9.2,8.0
...,...,...,...
95,"The question provided is: ""How do you think yo...",NaN,8.0
96,"""How do you perceive the alignment between you...",8.0,8.0
97,"""How do you plan to address any conflicts that...",8.2,8.4
98,Question Quality Score: 8\n\nThe question is c...,8.0,8.0


In [22]:
question_data.describe()

,openai_chain_question_text_quality_score,openai_chain_text_summary_quality_score,openai_chain_text_kobart_summary_quality_score,openai_chain_text_T5_summary_quality_score
count,89.000000,91.000000,90.000000,95.000000
mean,7.822472,7.830769,8.022222,7.871579
std,0.961858,0.622485,0.500287,0.592087
min,2.000000,6.000000,6.000000,5.000000
25%,8.000000,7.600000,8.000000,8.000000
50%,8.000000,8.000000,8.000000,8.000000
75%,8.200000,8.000000,8.350000,8.000000
max,9.200000,9.000000,9.200000,9.000000
